In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import fftpack
img = plt.imread('http://members.cbio.mines-paristech.fr/~nvaroquaux/formations/scipy-lecture-notes/_images/moonlanding.png')
img2_ft = fftpack.fft2(img, axes = (0,1))
mask = [abs(img2_ft) > 3100]
img2_ft[tuple(mask)] = 0
img2 = fftpack.ifft2(img2_ft, axes = (0,1)).real
plt.figure(figsize = [10,15])
plt.imshow(img)
plt.title('Not denoised image')
plt.show()
plt.figure(figsize = [10,15])
plt.imshow(img2)
plt.title('Denoised image')
plt.show()


___Exercise: Curve fitting of temperature data___

In [ ]:
from scipy.optimize import curve_fit
def f(x, a, b, c, d):
    return a * np.cos(b*x + c) + d
max_ = np.array([17, 19, 21, 28, 33, 38, 37,37, 31, 23, 19, 18])
min_ = np.array([-62,-59,-56,-46,-32,-18,-9,-13,-25,-46,-52,-58])
time = np.array([1,2,3,4,5,6,7,8,9,10,11,12])
guess = [60, 1/4, -6,-60]
params, params_covariance = curve_fit(f, time, min_, guess)
guess = [20, 1/4, -6, 20]
params_, params_covariance_ = curve_fit(f, time, max_, guess)
plt.figure(figsize = [10,15])
plt.plot(time,min_, label = "min")
plt.plot(time, f(time, params[0],params[1],params[2],params[3]), label = "min function")
plt.plot(time,max_, label = "max")
plt.plot(time, f(time, params_[0],params_[1],params_[2],params_[3]), label = "max function")
plt.legend()
plt.show()

___Exercise: 2-D minimization___

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
from scipy import optimize

def func(x):
    return (4 - 2.1 *x[0]*x[0]+x[0]*x[0]*x[0]*x[0]/3)*x[0]*x[0] + x[0]*x[1] + (4*x[1]*x[1]-4) * x[1]*x[1]
x = np.linspace(-2, 2)
y = np.linspace(-1, 1)
xg, yg = np.meshgrid(x, y)
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
surf = ax.plot_surface(xg, yg, func([xg, yg]), rstride=1, cstride=1,
                       cmap=plt.cm.jet, linewidth=0, antialiased=False)
rranges = (slice(-2, 2, 0.25), slice(-1, 1, 0.25))
resbrute = optimize.brute(func, rranges, full_output=True, finish=optimize.fmin)
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_zlabel('f(x, y)')
print("Global minimum = ", resbrute[0], "\n")
print("Function value = ", resbrute[1], "\n")

___1.5.11.13. Maximum wind speed prediction at the Sprogø station___

In [ ]:
import numpy as np
from scipy.interpolate import UnivariateSpline
import pylab as pl


def gumbell_dist(arr):
    return -np.log(-np.log(arr))

years_nb = 21
wspeeds = np.load('sprog-windspeeds.npy')
max_speeds = np.array([arr.max() for arr in np.array_split(wspeeds, years_nb)])
sorted_max_speeds = np.sort(max_speeds)

cprob = (np.arange(years_nb, dtype=np.float32) + 1)/(years_nb + 1)
gprob = gumbell_dist(cprob)
speed_spline = UnivariateSpline(gprob, sorted_max_speeds, k=1)
nprob = gumbell_dist(np.linspace(1e-3, 1-1e-3, 1e2))
fitted_max_speeds = speed_spline(nprob)

fifty_prob = gumbell_dist(49./50.)
fifty_wind = speed_spline(fifty_prob)

pl.figure()
pl.bar(np.arange(years_nb) + 1, max_speeds)
pl.axis('tight')
pl.xlabel('Year')

In [ ]:
import numpy as np
from scipy.interpolate import UnivariateSpline
import pylab as pl


def gumbell_dist(arr):
    return -np.log(-np.log(arr))

years_nb = 21
wspeeds = np.load('sprog-windspeeds.npy')
max_speeds = np.array([arr.max() for arr in np.array_split(wspeeds, years_nb)])
sorted_max_speeds = np.sort(max_speeds)

cprob = (np.arange(years_nb, dtype=np.float32) + 1)/(years_nb + 1)
gprob = gumbell_dist(cprob)
speed_spline = UnivariateSpline(gprob, sorted_max_speeds, k=1)
nprob = gumbell_dist(np.linspace(1e-3, 1-1e-3, 1e2))
fitted_max_speeds = speed_spline(nprob)

fifty_prob = gumbell_dist(49./50.)
fifty_wind = speed_spline(fifty_prob)

pl.figure()
pl.plot(sorted_max_speeds, gprob, 'o')
pl.plot(fitted_max_speeds, nprob, 'g--')
pl.plot([fifty_wind], [fifty_prob], 'o', ms=8., mfc='y', mec='y')
pl.plot([fifty_wind, fifty_wind], [pl.axis()[2], fifty_prob], 'k--')
pl.text(35, -1, r'$V_{50} = %.2f \, m/s$' % fifty_wind)
pl.xlabel('Annual wind speed maxima [$m/s$]')
pl.ylabel('Gumbell cumulative probability')
pl.show()

___1.5.11.16. Example of solution for the image processing exercise: unmolten grains in glass___

In [ ]:
import numpy as np
import pylab as pl
from scipy import ndimage
import matplotlib.pyplot as plt

In [ ]:
dat = pl.imread('MV_HFV_012.jpg')
dat = dat[60:]
filtdat = ndimage.median_filter(dat, size=(7,7))
hi_dat = np.histogram(dat, bins=np.arange(256))
hi_filtdat = np.histogram(filtdat, bins=np.arange(256))
#plt.hist(filtdat, bins=np.arange(256))
#plt.show()
void = filtdat <= 50
sand = np.logical_and(filtdat > 50, filtdat <= 114)
glass = filtdat > 114
phases = void.astype(np.int) + 2*glass.astype(np.int) + 3*sand.astype(np.int)
sand_op = ndimage.binary_opening(sand, iterations=2)
sand_labels, sand_nb = ndimage.label(sand_op)
sand_areas = np.array(ndimage.sum(sand_op, sand_labels, np.arange(sand_labels.max()+1)))
mask = sand_areas > 100
remove_small_sand = mask[sand_labels.ravel()].reshape(sand_labels.shape)
bubbles_labels, bubbles_nb = ndimage.label(void)
bubbles_areas = np.bincount(bubbles_labels.ravel())[1:]
mean_bubble_size = bubbles_areas.mean()
median_bubble_size = np.median(bubbles_areas)
mean_bubble_size, median_bubble_size